### scraping 데이터를 json화 해서 다루는 방법 & 정규표현식

In [1]:
import numpy as np
import pandas as pd


from datetime import date,datetime,timedelta
from dateutil.parser import parse

import json
import re
from glob import glob

# 시각화
%matplotlib inline
import seaborn as sns
import folium as g

import matplotlib.pyplot as plt
from matplotlib import rc
# 한글 폰트 문제 해결
rc('font', family='AppleGothic')
# 차트 축 <- 음수 부호 지원
plt.rcParams['axes.unicode_minus'] = False


# 크롤링
from bs4 import BeautifulSoup
from urllib.request import urlopen, urlretrieve
from urllib.error   import HTTPError
from urllib.error   import URLError

from selenium import webdriver

In [9]:
try : 
    html = urlopen('https://www.hanbit.co.kr/store/books/full_book_list.html')
    #print(html.read())
except HTTPError as he:
    print('http error')
except URLError as ue:
    print('url error')
else:
    # text = html.read().decode() 
    text = html.read().decode(html.info().get_content_charset('utf-8')) 

text # 문자열로 되어있다

'<!DOCTYPE html>\r\n<html lang="ko">\r\n<head>\r\n<!--[if lte IE 8]>\r\n<script>\r\n  location.replace(\'/support/explorer_upgrade.html\');\r\n</script>\r\n<![endif]-->\r\n<meta charset="utf-8"/>\r\n<title>한빛출판네트워크</title>\r\n<link rel="shortcut icon" href="https://www.hanbit.co.kr/images/common/hanbit.ico"> \r\n<meta http-equiv="X-UA-Compatible" content="IE=Edge" />\r\n<meta property="og:type" content="website"/>\r\n<meta property="og:title" content="한빛출판네트워크"/>\r\n<meta property="og:description" content="더 넓은 세상, 더 나은 미래를 위한 아시아 출판 네트워크 :: 한빛미디어, 한빛아카데미, 한빛비즈, 한빛라이프, 한빛에듀"/>\r\n<meta property="og:image" content="https://www.hanbit.co.kr/images/hanbitpubnet_logo.jpg" />\r\n<meta property="og:url" content="https://www.hanbit.co.kr/store/books/full_book_list.html"/>\r\n<link rel="canonical" href="https://www.hanbit.co.kr/store/books/full_book_list.html" />\r\n<meta name="keywords" content="한빛미디어,한빛아카데미,한빛비즈,한빛라이프,한빛에듀,리얼타임,대관서비스,책,출판사,IT전문서,IT활용서,대학교재,경제경영,어린이/유아,실용/여행,전자책,자격증,교육,세미나,강의

#### 데이터를 json 형식으로 인코딩 
- josn.dumps() : 문자열
- json.dump() : 파일

### 메타문자
- []: 괄호 사이의 문자들과 매치
- -(하이픈): [from -  to]

#
- \d: 숫자[0-9]
- \D: 숫자가 아닌것을 의미[^0-9]
- \s: 공백[\n\t\r\v]
- \S: 공백이 아닌것[^\n\t\r\v]
- \w: 문자+숫자+_ [a-zA-Z0-9_]
- \W: 문자+숫자+_ 아닌것 [^a-zA-Z0-9_]
- . : 개행문자(\n)를 제외한 모든 문자
    
#
- ^[]: 시작문자를 매치        ex) ^a
- $[]: 문자열의 끝과 매치     ex) $a

#
- * : 반복(0-무한대)
- + : 반복(1-무한대)
- ? : 있어도 되고 없어도 된다.(0 or 1)
- {cnt} : cnt만큼 반복        ex) {x,y}: x~y번 반복

### 정규표현식
- match() : 문자의 처음부터 정규식과 매치되는지를 검색
- search() : 문자 전체에서 정규식과 매치되는 부분이 있는지를 검색
- findall() : 정규식과 매치되는 모든 문자를 찾아서 리스트로 반환
- finditer() : 정규식과 매치되는 모든 문자를 찾아서 반복가능한 객체로 변환
- sub() : 정규식과 매치되는 부분을 치환

In [13]:
re.compile(r'[apple]').findall('I ate apple')

['a', 'e', 'a', 'p', 'p', 'l', 'e']

In [15]:
re.compile(r'[1-3]').findall('123115325215135246426347')

['1', '2', '3', '1', '1', '3', '2', '2', '1', '1', '3', '2', '2', '3']

In [16]:
re.compile(r'[a-e]').findall('albiaksaldjfiae3nai3naklsif')

['a', 'b', 'a', 'a', 'd', 'a', 'e', 'a', 'a']

In [17]:
re.compile(r'\d').findall('albiaksaldjfiae3nai3naklsif')

['3', '3']

In [21]:
re.compile(r'\D').findall('23gasif')

['g', 'a', 's', 'i', 'f']

In [19]:
re.compile(r'\s').findall('I like\t you\n')

[' ', '\t', ' ', '\n']

In [25]:
re.search(r'\s','I like\t you\n')

<re.Match object; span=(1, 2), match=' '>

In [62]:
re.findall(r'<td class="left">.*</td>',text)

['<td class="left"><a href="/store/books/look.php?p_code=B6770191984">디자인 1등 유튜버 존코바의 클릭을 유도하는 유튜브 디자인 with 포토샵</a></td>',
 '<td class="left">존코바(이요한)   </td>',
 '<td class="left"><a href="/store/books/look.php?p_code=B2393470870">두 발로 선 경제</a></td>',
 '<td class="left">이용우   </td>',
 '<td class="left"><a href="/store/books/look.php?p_code=B4517369064">인공지능 논리 입문</a></td>',
 '<td class="left">이은정   </td>',
 '<td class="left"><a href="/store/books/look.php?p_code=B8274600462">머신러닝 파워드 애플리케이션</a></td>',
 '<td class="left">에마뉘엘 아메장   </td>',
 '<td class="left"><a href="/store/books/look.php?p_code=B9363843967">유튜브 마케팅 인사이트</a></td>',
 '<td class="left">서양수   </td>',
 '<td class="left"><a href="/store/books/look.php?p_code=B4448108740">웹 브라우저 속 머신러닝 TensorFlow.js</a></td>',
 '<td class="left">카이 사사키   </td>',
 '<td class="left"><a href="/store/books/look.php?p_code=B6434597961">IT CookBook, 처음 만나는 전기기기&#40;2판&#41;</a></td>',
 '<td class="left">최은혁 외 1명  </td>',
 '<td class="left"><a href="

In [48]:
re.findall('<td class="left"><a.*></td>',text)

['<td class="left"><a href="/store/books/look.php?p_code=B6770191984">디자인 1등 유튜버 존코바의 클릭을 유도하는 유튜브 디자인 with 포토샵</a></td>',
 '<td class="left"><a href="/store/books/look.php?p_code=B2393470870">두 발로 선 경제</a></td>',
 '<td class="left"><a href="/store/books/look.php?p_code=B4517369064">인공지능 논리 입문</a></td>',
 '<td class="left"><a href="/store/books/look.php?p_code=B8274600462">머신러닝 파워드 애플리케이션</a></td>',
 '<td class="left"><a href="/store/books/look.php?p_code=B9363843967">유튜브 마케팅 인사이트</a></td>',
 '<td class="left"><a href="/store/books/look.php?p_code=B4448108740">웹 브라우저 속 머신러닝 TensorFlow.js</a></td>',
 '<td class="left"><a href="/store/books/look.php?p_code=B6434597961">IT CookBook, 처음 만나는 전기기기&#40;2판&#41;</a></td>',
 '<td class="left"><a href="/store/books/look.php?p_code=B9923613168">모바일 UX/UI 디자인 강의 with Adobe XD</a></td>',
 '<td class="left"><a href="/store/books/look.php?p_code=B9575488572">처음 배우는 플라스크 웹 프로그래밍</a></td>',
 '<td class="left"><a href="/store/books/look.php?p_code=B92346

In [73]:
re.search(r'<td class="left"><a.*?>(.*)</td>',text).group()

'<td class="left"><a href="/store/books/look.php?p_code=B6770191984">디자인 1등 유튜버 존코바의 클릭을 유도하는 유튜브 디자인 with 포토샵</a></td>'

In [67]:
re.findall(r'<td class="left"><a.*?>(.*)</td>',text)

['디자인 1등 유튜버 존코바의 클릭을 유도하는 유튜브 디자인 with 포토샵</a>',
 '두 발로 선 경제</a>',
 '인공지능 논리 입문</a>',
 '머신러닝 파워드 애플리케이션</a>',
 '유튜브 마케팅 인사이트</a>',
 '웹 브라우저 속 머신러닝 TensorFlow.js</a>',
 'IT CookBook, 처음 만나는 전기기기&#40;2판&#41;</a>',
 '모바일 UX/UI 디자인 강의 with Adobe XD</a>',
 '처음 배우는 플라스크 웹 프로그래밍</a>',
 '인문학 거저보기 : 서양철학 편</a>',
 'IT CookBook, 현대 센서공학&#40;5판&#41;</a>',
 '방구석 심리학 실험실</a>',
 'IT CookBook, 모바일 웹 서비스 구현을 위한 JSP 웹 프로그래밍</a>',
 '혼자서도 척척 접는 종이접기</a>',
 '혼자서도 척척 접는 종이접기 플러스</a>',
 '혼자서도 척척 접는 종이접기 2권 세트</a>',
 '스파크를 이용한 자연어 처리</a>',
 '클라우드 네이티브를 위한 데이터 센터 네트워크 구축</a>',
 '어른을 위한 두뇌 놀이 책</a>',
 '감으로만 일하던 김 팀장은 어떻게 데이터 좀 아는 팀장이 되었나</a>',
 '쉽게 배우고 익히는 회계원리[객관식/학습문제 해답]&#40;4판&#41;</a>',
 '미래조직을 위한 조직설계의 이해&#40;4판&#41;</a>',
 '쉽게 배우고 익히는 회계원리&#40;4판&#41;</a>',
 'fastai와 파이토치가 만나 꽃피운 딥러닝</a>',
 '스케치업 & V-Ray 트레이닝 북&#40;개정판&#41;</a>',
 'IT CookBook, 짧고 굵게 배우는 JSP 웹 프로그래밍과 스프링 프레임워크</a>',
 '생각과 말로 글을 늘리는 초등 10분 글쓰기</a>',
 '만들면서 배우는 파이토치 딥러닝</a>',
 '데이터베이스 시스템&#40;7판&#41;</a>',
 '실리콘밸리 리더십</a>',
 '처음 시작하는 마이크로

In [50]:
tds = re.findall('<td class="left"><a.*?></td>',text)    # 이거 존나 이해가 안간다
for td in tds:
    #print(td)
    print(re.search(r'<a href="(.*)">(.*)</a>',td).group(1))
    print(re.search(r'<a href="(.*)">(.*)</a>',td).group(2))
    

/store/books/look.php?p_code=B6770191984
디자인 1등 유튜버 존코바의 클릭을 유도하는 유튜브 디자인 with 포토샵
/store/books/look.php?p_code=B2393470870
두 발로 선 경제
/store/books/look.php?p_code=B4517369064
인공지능 논리 입문
/store/books/look.php?p_code=B8274600462
머신러닝 파워드 애플리케이션
/store/books/look.php?p_code=B9363843967
유튜브 마케팅 인사이트
/store/books/look.php?p_code=B4448108740
웹 브라우저 속 머신러닝 TensorFlow.js
/store/books/look.php?p_code=B6434597961
IT CookBook, 처음 만나는 전기기기&#40;2판&#41;
/store/books/look.php?p_code=B9923613168
모바일 UX/UI 디자인 강의 with Adobe XD
/store/books/look.php?p_code=B9575488572
처음 배우는 플라스크 웹 프로그래밍
/store/books/look.php?p_code=B9234684497
인문학 거저보기 : 서양철학 편
/store/books/look.php?p_code=B4633012179
IT CookBook, 현대 센서공학&#40;5판&#41;
/store/books/look.php?p_code=B9037188841
방구석 심리학 실험실
/store/books/look.php?p_code=B1265976064
IT CookBook, 모바일 웹 서비스 구현을 위한 JSP 웹 프로그래밍
/store/books/look.php?p_code=B6220676898
혼자서도 척척 접는 종이접기
/store/books/look.php?p_code=B9134095146
혼자서도 척척 접는 종이접기 플러스
/store/books/look.php?p_code=B615683

In [58]:
# ensure_ascii
# indent
with open('./Data/book_list.json','w',encoding='utf-8') as file:
    data = []
    tds = re.findall('<td class="left"><a.*?></td>',text)
    for td in tds:
        url = 'https://www.hanbit.co.kr' + re.search(r'<a href="(.*)">(.*)</a>',td).group(1)
        title = re.search(r'<a href="(.*)">(.*)</a>',td).group(2)
        data.append({'bookname':title,'url':url})
    json.dump(data,file,ensure_ascii=False,indent=2)

## 실습
- www.cine21.com/rank/person
1. 라이브러리 import 
2. db connection - db create - collection create
3. request
4. 현재 페이지의 배우 이름만 가져오기(특수문자 제외)
5. 현재 페이지에서 클릭후 상세정보 가져오기; 배우이름,흥행지수,출연영화,랭킹,원어명,다른이름,직업,생년월일,성별,홈페이지,신장/체중,학교,취미,특기
6. 배우이름,흥행지수,출연영화,랭킹 db에 저장
7. page처리